In [78]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter, FuncAnimation
import os
from scipy.spatial import distance
from scipy.ndimage import zoom

# Parameters
image_max = (500, 500)  # pixel (height, width)
scale_reduction = 5  # size reduction for antialiasing
n_frames = 100  # number of frames
n_seeds = 15 # number of seeds

# Define versions and their corresponding p values
version_list = {
    'minkowski_p0_125': 0.125,
    'minkowski_p0_707': 2 ** -0.5,
    'cityblock': 1,
    'minkowski_p1_25': 1.25,
    'euclidean': 2,
    'minkowski_p3': 3,
    'chebychev': np.inf,
    'negative_chebyshev': -np.inf, 
    'minkowski_p100': 100  
} # 'minkowski_p_minus3': -3, 'minkowski_p_minus10': -10, 

# Minkowski distance function
def minkowski_distances(a, b, p):
    return distance.cdist(a, b, metric='minkowski', p=p)

# Helper function to create Voronoi growth
def voronoi_growth(seeds, p, n_frames, image_max, scale_reduction):
    seeds = (seeds * np.array(image_max)).astype(int) * scale_reduction
    X, Y = np.meshgrid(np.arange(image_max[1] * scale_reduction), np.arange(image_max[0] * scale_reduction))
    coordinates = np.c_[X.ravel(), Y.ravel()]
    
    # Calculate distance matrix
    distances = minkowski_distances(coordinates, seeds, p)
    min_distances = distances.min(axis=1)
    min_point_indices = distances.argmin(axis=1)
    
    # Initialize color index array
    color_indices = np.full_like(min_point_indices, n_seeds + 1)
    reduced_rgb_images = np.ones((image_max[0], image_max[1], 3, n_frames))
    
    # Growth steps from start point to full coverage
    distance_point = image_max[0] * scale_reduction * (0.01 + 0.0012 / p ** 3)
    distance_list = np.linspace(distance_point, min_distances.max() + np.finfo(float).eps, n_frames)
    
    for i_frame, current_growth_dist in enumerate(distance_list):
        color_pixel = min_distances <= current_growth_dist
        
        if i_frame == 0:
            color_indices[color_pixel] = n_seeds
        else:
            color_indices[color_pixel] = min_point_indices[color_pixel]
        
        min_distances[color_pixel] = np.inf
        
        # Create RGB image for the current frame
        raw_rgb_image = colors[color_indices].reshape(image_max[0] * scale_reduction, image_max[1] * scale_reduction, 3)
        
        # Reduce size and add antialiasing
        temp_image = zoom(raw_rgb_image, (1/scale_reduction, 1/scale_reduction, 1))
        
        # Clip values to the valid range [0, 1]
        temp_image = np.clip(temp_image, 0, 1)
        
        reduced_rgb_images[..., i_frame] = temp_image
    
    return reduced_rgb_images

# Create output directory
output_dir = 'voronoi_growth'
os.makedirs(output_dir, exist_ok=True)

# Define color map for Voronoi cells using distinct colors
colormap = plt.cm.Spectral  # Distinct colors
colors = colormap(np.linspace(0, 1, n_seeds))[:, :3]  # Use only RGB channels
colors = np.vstack((colors, [0, 0, 0], [1, 1, 1]))  # Add colors for initial state and background

# Ensure colors are in the range [0, 1]
colors = np.clip(colors, 0, 1)

# Sampling options
sampling_methods = {
    'uniform': lambda n: np.random.rand(n, 2),
    'normal': lambda n: np.clip(np.random.normal(0.5, 1/6, (n, 2)), 0, 1),
    'exponential': lambda n: np.clip(np.random.exponential(0.5, (n, 2)), 0, 1)
}

# Generate Voronoi growth for each version and sampling method
for sampling_name, sampling_method in sampling_methods.items():
    seeds_def = sampling_method(n_seeds)
    
    for version_name, p in version_list.items():
        images = voronoi_growth(seeds_def, p, n_frames, image_max, scale_reduction)
        
        # Create a figure
        fig, ax = plt.subplots()
        ax.set_xlim(0, image_max[1])
        ax.set_ylim(0, image_max[0])
        ax.set_xticks([])
        ax.set_yticks([])
        
        def update(frame):
            ax.imshow(images[..., frame])
            ax.set_title(f'{sampling_name} - {version_name} - Frame {frame + 1}/{n_frames}')
        
        ani = FuncAnimation(fig, update, frames=n_frames, interval=50, repeat=False)
        ani.save(os.path.join(output_dir, f'Voronoi_growth_{sampling_name}_{version_name}.gif'), writer=PillowWriter(fps=20))
        
        plt.close(fig)

print("Voronoi growth GIFs have been generated.")




/Users/borister-avanesov/opt/anaconda3/lib/python3.8/site-packages/numpy/core/function_base.py:157: RuntimeWarning: invalid value encountered in multiply
  y *= step


Voronoi growth GIFs have been generated.
